In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import numpy as np
import os.path as osp
from evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'
do_jsd = False
do_mmd = True
save_res = True
top_out_dir = top_evaluation_dir

experiment_name = 'gmm'
sample_mmd = True

n_samples = 10
ref_pop = 40
sample_pop = 200

mmd_loss = 'emd'

special_tag = 'emd_sample_40_200'
skip = ['test', 'val']

In [5]:
top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]

In [6]:
if save_res:
    exp_name = osp.basename(top_sample_dir)
    if do_jsd:
        f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + exp_name + '.txt'), 'w')
            
    if do_mmd:
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
else:
    f_jsd = sys.stdout
    f_mmd = sys.stdout

In [7]:
if experiment_name.startswith('l_') and 'gan' in experiment_name:
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 2001, 100)])
elif experiment_name.startswith('r_gan'):
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])

In [8]:
sample_files = []
if 'gan' in experiment_name:
    for epoch in epochs_to_check:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
elif 'gmm' in experiment_name:
    for f in files_in_subdirs(top_sample_dir, '.npz'):
        sample_files.append(f) 
else:
    assert False

In [9]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [10]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)
        f_mmd.flush()
    print

bneck_128_diag_20_gaussians
train 276.272 4.9693

bneck_128_full_40_gaussians
train 289.014 7.91694

bneck_128_full_24_gaussians
train 283.632 7.34627

bneck_128_full_30_gaussians
train 285.031 4.84107

bneck_128_full_18_gaussians
train 291.191 12.5011

bneck_128_full_12_gaussians
train 292.199 7.19675

bneck_128_full_4_gaussians
train 283.824 5.81224

bneck_128_full_10_gaussians
train 288.307 8.04031

bneck_128_full_38_gaussians
train 291.367 9.17959

bneck_128_full_6_gaussians
train 283.556 5.16382

bneck_128_full_26_gaussians
train 282.526 6.89303

bneck_128_full_32_gaussians
train 293.174 5.29465

bneck_128_full_36_gaussians
train 289.773 10.0115

bneck_128_full_8_gaussians
train 292.737 7.18419

bneck_128_full_22_gaussians
train 293.03 6.73058

bneck_128_full_14_gaussians
train 287.803 7.67572

bneck_128_full_2_gaussians
train 284.646 6.81409

bneck_128_full_28_gaussians
train 290.784 7.30619

bneck_128_diag_10_gaussians
train 269.332 3.96571

bneck_128_diag_5_gaussians
train 263.

In [11]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()